# Summary
Given a trade represented by the ticker symbol, date and match_id, get the trade's exchange_timestamp and enrich with BBO (from certain venues) and NBBO just before the trade 

Add the necessary imports and define a convenience function

In [ ]:
import datetime
import hpq
import ijson
import IPython.display
import numpy
import tabulate


def transform_nbbo(obj):
    return {
        "bid_price": obj["bid_price"],
        "bid_quantity": obj["bid_quantity"],
        "ask_price": obj["ask_price"],
        "ask_quantity": obj["ask_quantity"],
    }

A trade denoted by the security, match_id and approximate time of the trade is received. All times in UTC.

In [ ]:
match_id_of_trade = 52983542269494
security = "XOM"
apprx_date_time_of_trade = datetime.datetime(2023, 1, 10, 17, 10, 50, 40400)
print(apprx_date_time_of_trade)

Query for trades starting from 15 minutes before the approximate trade time to the approximate trade time. Identify the trade using match_id and store the exchange_timestamp of the trade

In [ ]:
connection = hpq.create_web_socket_client()
request = {
    "query": "ticks",
    "messages": "TRD",
    "source": "sip",
    "product": security,
    "time_zone": "UTC",
    "date": apprx_date_time_of_trade.strftime("%Y-%m-%d"),
    "start_time": (apprx_date_time_of_trade - datetime.timedelta(minutes=15)).strftime(
        "%H:%M:%S.%f"
    ),
    "end_time": apprx_date_time_of_trade.strftime("%H:%M:%S.%f"),
}

trades = ijson.items(connection.stream(request), "item")
trade_exch_timestamp = 0
for item in trades:
    if item["match_id"] == match_id_of_trade:
        trade_exch_timestamp = item["exchange_timestamp"]

Request a snapshot of bbo_quotes from 1 nanosecond before the trade's exchange_timestamp

In [ ]:
trade_exch_timestamp -= 1
trade_date_time = numpy.datetime64(trade_exch_timestamp, "ns")

snap_request = {
    "query": "snapshot",
    "type": "bbo_quotes",
    "source": "sip[nyse,nysearca,bats,nasdaqomx]",
    "product": "XOM",
    "time_zone": "UTC",
    "date": numpy.datetime_as_string(trade_date_time, "D", "UTC"),
    "time": trade_date_time.astype(str)[11:29],
}

snapshot = ijson.items(connection.stream(snap_request), "item")


def transform_bbo(obj):
    return {"market_participant": obj["market_participant"], **transform_nbbo(obj)}


data = list(map(transform_bbo, snapshot))
table = tabulate.tabulate(data, tablefmt="html", headers="keys")
IPython.display.display(IPython.display.HTML("<h1>BBOs</h1>"))
IPython.display.display(IPython.display.HTML(table))

Request a snapshot of the nbbo quote

In [ ]:
snap_request["type"] = "nbbo_quotes"
snap_request["source"] = "sip"
nbbo_snapshot = ijson.items(connection.stream(snap_request), "item")
data = list(map(transform_nbbo, nbbo_snapshot))
table = tabulate.tabulate(data, tablefmt="html", headers="keys")
IPython.display.display(IPython.display.HTML("<h1>NBBO</h1>"))
IPython.display.display(IPython.display.HTML(table))